### Notebook for concept detection in neural network

In [15]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

from concepts import static_concepts, linear_regression, generate_static_concept_datasets
from policy import ConvNet, ResNet
from utils import concept_folder_setup_and_score

import tensorflow as tf

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

reward_function = "zero_sum" # "concept_fn", "zero_sum" or "jem"
model_name = "net"
session_name = "probing"
board_size = 7
board_name = f'{board_size}x{board_size}'
resnet = True

model_type = "resnet" if resnet else "convnet"

agents_to_sample = [0, 10, 50, 100, 300, 500, 600, 800, 1000]

full_model_path = f"../models/saved_sessions/{model_type}/{reward_function}/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.win_on_pass

CONCEPT_NAME = static_concepts.win_on_pass.__name__

BINARY = True

CASES_TO_SAMPLE = 2500 # 25000

SAMPLE_RATIO = 0.8

# Load the models
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, model_path)
    else:
        model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [2]:
positive_cases, negative_cases = generate_static_concept_datasets(CASES_TO_SAMPLE, agents, board_size, CONCEPT_FUNC, sample_ratio=SAMPLE_RATIO, binary=BINARY)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases for concept 'win_on_pass':   0%|          | 0/2500 [00:00<?, ?it/s]

Positive cases for concept 'win_on_pass': 2521it [40:00,  1.05it/s]                           


In [3]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 5, 7, 7)
Negative cases:  (2500, 5, 7, 7)


In [4]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [3]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  15


In [ ]:
from env import gogame
# Print all the positive cases
for i in range(100, 120):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [6]:
# Positions to consider are 80% of the total positions
if BINARY:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000 #10000
else:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0])
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000

epochs = 5

Positions to consider: 4000


In [5]:
# First test if the concept can be regressed form the inputs
name = "input"
if BINARY:
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
else:
    all_cases = np.array(positive_cases)
    all_labels = negative_cases

all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs,
    verbose=1
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, name, score)

2024-04-24 22:08:49.747055: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


125/125 [==============================] - 0s 3ms/step
0.0015000000000000568
32/32 [==============================] - 0s 5ms/step
Regression score:  0.02400000000000002


In [7]:
from tqdm import tqdm
epochs_to_look_at = agents_to_sample

bar = tqdm(total=len(epochs_to_look_at), desc="Epochs")

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    if BINARY:
        all_cases = np.concatenate([positive_cases, negative_cases])
        all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    else:
        all_cases = np.array(positive_cases)
        all_labels = negative_cases

    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs,
            verbose=1
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
        
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, epoch, concept_presence_per_layer)

    bar.update()

Getting activation outputs: 100%|██████████| 157/157 [00:04<00:00, 32.95it/s]


Performing regression for layer 0
Epoch 1/5


2024-04-24 22:09:09.425471: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


125/125 [==============================] - 3s 19ms/step - loss: 1.3069 - val_loss: 1.1203
Epoch 2/5
125/125 [==============================] - 2s 16ms/step - loss: 0.9875 - val_loss: 0.8638
Epoch 3/5
125/125 [==============================] - 2s 16ms/step - loss: 0.7888 - val_loss: 0.7262
Epoch 4/5
125/125 [==============================] - 2s 16ms/step - loss: 0.7063 - val_loss: 0.6881
Epoch 5/5
125/125 [==============================] - 0s 3ms/step
0.5165
32/32 [==============================] - 0s 3ms/step
The presence of win_on_pass in resblock 0 is 0.556
Performing regression for layer 1
Epoch 1/5


2024-04-24 22:09:20.982270: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

125/125 [==============================] - 3s 21ms/step - loss: 8.0059 - val_loss: 6.7963
Epoch 2/5
125/125 [==============================] - 2s 17ms/step - loss: 7.1059 - val_loss: 6.1863
Epoch 3/5
125/125 [==============================] - 2s 17ms/step - loss: 6.3486 - val_loss: 5.6739
Epoch 4/5
125/125 [==============================] - 2s 17ms/step - loss: 5.7182 - val_loss: 5.2421
Epoch 5/5
125/125 [==============================] - 0s 2ms/step
0.1915
32/32 [==============================] - 0s 3ms/step
The presence of win_on_pass in resblock 6 is 0.248
Performing regression for layer 7
Epoch 1/5


2024-04-24 22:24:53.653858: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


125/125 [==============================] - 3s 20ms/step - loss: 9.3824 - val_loss: 9.1128
Epoch 2/5
125/125 [==============================] - 2s 17ms/step - loss: 8.0420 - val_loss: 7.9227
Epoch 3/5
125/125 [==============================] - 2s 17ms/step - loss: 6.9491 - val_loss: 6.9669
Epoch 4/5
125/125 [==============================] - 2s 17ms/step - loss: 6.1098 - val_loss: 6.2555
Epoch 5/5
125/125 [==============================] - 0s 2ms/step
0.030999999999999917
32/32 [==============================] - 0s 3ms/step


Epochs: 100%|██████████| 9/9 [16:02<00:00, 109.82s/it]

The presence of win_on_pass in resblock 7 is 0.008000000000000007
